# Tutorial: Ingeniería de Datos para Análisis de Clientes

## Introducción

Este notebook se centra en la ingeniería de datos para construir un conjunto de datos analítico a nivel de cliente. Partiendo de los datos de facturación (`df_analisis.parquet`) y la información de clientes (`clientes.parquet`), crearemos nuevas características que resuman el comportamiento y las propiedades de cada cliente. Este tipo de dataset es fundamental para modelos predictivos más sofisticados, segmentación avanzada, y análisis del valor del cliente (CLV).

**Objetivos:**
1.  Cargar los datos preprocesados.
2.  Realizar ingeniería de características a partir de datos transaccionales (facturas) para crear una vista agregada por cliente.
3.  Incorporar características estáticas del cliente.
4.  Manejar variables categóricas y asegurar la calidad de los datos.
5.  Crear un DataFrame final listo para análisis avanzados o modelado.

## 1. Configuración del Entorno y Carga de Datos

### 1.1 Importación de Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 7)

### 1.2 Descarga y Carga de los Datos Base

Cargaremos `df_analisis.parquet` (que contiene la información detallada de facturas y pagos) y `clientes.parquet` (para datos demográficos y contractuales estáticos del cliente).

In [ ]:
!wget -N https://github.com/srJboca/segmentacion/raw/refs/heads/main/archivos/df_analisis.parquet
!wget -N https://github.com/srJboca/segmentacion/raw/refs/heads/main/archivos/clientes.parquet

df_analisis = pd.read_parquet('df_analisis.parquet')
df_clientes = pd.read_parquet('clientes.parquet')

## 2. Revisión Inicial y Preparación de `df_analisis`

Asegurémonos de que las columnas de fecha estén en el formato correcto y realicemos cualquier ajuste necesario en `df_analisis` antes de la agregación.

In [ ]:
print("--- Información de df_analisis ---")
df_analisis.info()

# Columnas de fecha a convertir (asegurando que ya se hizo o haciéndolo ahora)
date_cols_analisis = ['Fecha de Emision', 'Fecha de Pago Oportuno', 'Fecha de Lectura', 
                        'Fecha de Suspension Estimada', 'Fecha de Pago Real']
for col in date_cols_analisis:
    if col in df_analisis.columns:
        df_analisis[col] = pd.to_datetime(df_analisis[col], errors='coerce')

# Eliminar la columna 'mora' si existe 'Mora' (artefacto de notebooks anteriores)
if 'mora' in df_analisis.columns and 'Mora' in df_analisis.columns:
    df_analisis.drop(columns=['mora'], inplace=True)
elif 'mora' in df_analisis.columns and 'Mora' not in df_analisis.columns:
    df_analisis.rename(columns={'mora': 'Mora'}, inplace=True)

print("\n--- df_analisis (primeras filas después de ajustes) ---")
print(df_analisis.head())

## 3. Ingeniería de Características a Nivel de Cliente (Agregación)

Crearemos un DataFrame donde cada fila represente un cliente único (`Numero de contrato`). Las columnas serán características agregadas que describan su comportamiento histórico.

**Características a calcular:**
* **Frecuencia:** Número total de facturas.
* **Monetario:**
    * Consumo total y promedio de `Consumo (m3)`.
    * Gasto total y promedio (`Precio por Consumo`).
    * Consumo mínimo y máximo.
    * Desviación estándar del consumo (variabilidad).
* **Comportamiento de Pago:**
    * Promedio de `Dias_Emision_PagoOportuno`.
    * Promedio de `Dias_Lectura_Emision`.
    * Promedio de `Dias_PagoOportuno_PagoReal` (días de mora/adelanto promedio).
    * Número total de facturas pagadas con mora.
    * Tasa de mora (proporción de facturas pagadas con mora).
    * Número de facturas sin pago registrado.
* **Recencia:** (Aunque no se calcula explícitamente la fecha de la última factura aquí, podríamos inferirla indirectamente o calcularla si fuera necesario para un modelo de recencia específico).

In [ ]:
df_clientes_agg = df_analisis.groupby('Numero de contrato').agg(
    Total_Facturas=('Numero de factura', 'count'),
    Consumo_Total_m3=('Consumo (m3)', 'sum'),
    Consumo_Promedio_m3=('Consumo (m3)', 'mean'),
    Consumo_Min_m3=('Consumo (m3)', 'min'),
    Consumo_Max_m3=('Consumo (m3)', 'max'),
    Consumo_Std_m3=('Consumo (m3)', 'std'),
    Gasto_Total=('Precio por Consumo', 'sum'),
    Gasto_Promedio=('Precio por Consumo', 'mean'),
    Dias_Emision_PagoOportuno_Promedio=('Dias_Emision_PagoOportuno', 'mean'),
    Dias_Lectura_Emision_Promedio=('Dias_Lectura_Emision', 'mean'),
    Dias_Pago_Vs_Oportuno_Promedio=('Dias_PagoOportuno_PagoReal', 'mean'), # Puede ser NaN si no hay pagos
    Total_Facturas_Mora=('Mora', 'sum'), # Suma de 1s donde Mora es True
    Facturas_Sin_Pago_Registrado=('Fecha de Pago Real', lambda x: x.isnull().sum())
).reset_index()

# Calcular Tasa de Mora
df_clientes_agg['Tasa_Mora'] = df_clientes_agg['Total_Facturas_Mora'] / df_clientes_agg['Total_Facturas']
df_clientes_agg['Tasa_Mora'].fillna(0, inplace=True) # Si no hay facturas, o no hay moras, tasa es 0

print("--- df_clientes_agg (primeras filas) ---")
print(df_clientes_agg.head())
print("\n--- Información de df_clientes_agg ---")
df_clientes_agg.info()

## 4. Enriquecimiento con Datos Estáticos del Cliente

Ahora, uniremos `df_clientes_agg` con `df_clientes` para añadir información demográfica y contractual como 'Ciudad', 'Estrato socioeconomico', y 'Fecha de Inicio del contrato'.

In [ ]:
df_clientes_seleccion = df_clientes[[
    'Numero de contrato', 'Ciudad', 'Estrato socioeconomico', 
    'Fecha de Inicio del contrato', 'Fecha de la Ultima revision periodica'
]].copy()

# Convertir fechas en df_clientes_seleccion
df_clientes_seleccion['Fecha de Inicio del contrato'] = pd.to_datetime(df_clientes_seleccion['Fecha de Inicio del contrato'], errors='coerce')
df_clientes_seleccion['Fecha de la Ultima revision periodica'] = pd.to_datetime(df_clientes_seleccion['Fecha de la Ultima revision periodica'], errors='coerce')

# Unir los DataFrames
df_clientes_completo = pd.merge(df_clientes_agg, df_clientes_seleccion, on='Numero de contrato', how='left')

print("--- df_clientes_completo (primeras filas) ---")
print(df_clientes_completo.head())
print("\n--- Información de df_clientes_completo ---")
df_clientes_completo.info()

### 4.1 Ingeniería de Características Basadas en Fechas del Cliente

* **Antigüedad del Cliente:** Tiempo desde el inicio del contrato hasta una fecha de referencia (e.g., la fecha más reciente en los datos o la fecha actual).
* **Tiempo desde Última Revisión:** Similar, para la fecha de última revisión.

In [ ]:
# Fecha de referencia (podría ser la fecha máxima de emisión de factura o la fecha actual)
fecha_referencia = df_analisis['Fecha de Emision'].max() 
if pd.isna(fecha_referencia):
    fecha_referencia = pd.Timestamp.now() # Fallback si no hay fechas de emisión
print(f"Fecha de referencia para cálculos de antigüedad: {fecha_referencia}")

df_clientes_completo['Antiguedad_Cliente_Dias'] = (fecha_referencia - df_clientes_completo['Fecha de Inicio del contrato']).dt.days
df_clientes_completo['Tiempo_Desde_Ult_Revision_Dias'] = (fecha_referencia - df_clientes_completo['Fecha de la Ultima revision periodica']).dt.days

print("\n--- df_clientes_completo con características de antigüedad ---")
print(df_clientes_completo[['Numero de contrato', 'Antiguedad_Cliente_Dias', 'Tiempo_Desde_Ult_Revision_Dias']].head())

## 5. Codificación de Variables Categóricas

Convertiremos variables categóricas como 'Estrato socioeconomico' y 'Ciudad' a un formato numérico para que puedan ser usadas por modelos de Machine Learning.

In [ ]:
# Codificación Ordinal para 'Estrato socioeconomico'
if df_clientes_completo['Estrato socioeconomico'].dtype == 'object' or isinstance(df_clientes_completo['Estrato socioeconomico'].dtype, pd.CategoricalDtype):
    df_clientes_completo['Estrato_Num'] = df_clientes_completo['Estrato socioeconomico'].str.replace('Estrato ', '', regex=False).astype(int)
else:
    df_clientes_completo['Estrato_Num'] = df_clientes_completo['Estrato socioeconomico'].astype(int)

# Codificación One-Hot para 'Ciudad' (si hay pocas categorías, de lo contrario considerar otras técnicas)
if df_clientes_completo['Ciudad'].nunique() < 10: # Umbral arbitrario para decidir One-Hot
    df_clientes_completo = pd.get_dummies(df_clientes_completo, columns=['Ciudad'], prefix='Ciudad')
else:
    # Para muchas ciudades, podría usarse Label Encoding (con precaución) o técnicas más avanzadas
    print("Muchas categorías en 'Ciudad', se omitirá One-Hot encoding por simplicidad en este tutorial.")
    # Podríamos usar Label Encoding como alternativa simple:
    # df_clientes_completo['Ciudad_Cod'] = df_clientes_completo['Ciudad'].astype('category').cat.codes

print("\n--- df_clientes_completo después de codificación ---")
print(df_clientes_completo.head())
df_clientes_completo.info()

## 6. Selección Final de Características y Limpieza

Seleccionaremos las columnas numéricas finales y manejaremos cualquier valor NaN restante (por ejemplo, por promedios de días de pago si un cliente nunca pagó, o desviación estándar si solo tuvo una factura).

In [ ]:
df_clientes_agg_final = df_clientes_completo.copy()

# Eliminar columnas de identificadores o fechas originales si no se usarán directamente
cols_to_drop = ['Numero de contrato', 'Estrato socioeconomico', 
                'Fecha de Inicio del contrato', 'Fecha de la Ultima revision periodica']
# También eliminar 'Ciudad' original si se hizo one-hot encoding
if 'Ciudad' in df_clientes_agg_final.columns and any(col.startswith('Ciudad_') for col in df_clientes_agg_final.columns):
    cols_to_drop.append('Ciudad')

df_clientes_agg_final.drop(columns=cols_to_drop, errors='ignore', inplace=True)

# Manejo de NaNs restantes
# Por ejemplo, 'Consumo_Std_m3' puede ser NaN si solo hay una factura (std de un solo valor es NaN)
# 'Dias_Pago_Vs_Oportuno_Promedio' puede ser NaN si no hay pagos registrados
df_clientes_agg_final['Consumo_Std_m3'].fillna(0, inplace=True) # Asumir 0 variabilidad si hay 1 factura

# Para 'Dias_Pago_Vs_Oportuno_Promedio', una estrategia podría ser imputar con un valor alto (si indica mal pagador)
# o la media/mediana del dataset, o 0 si se asume pago oportuno por defecto para no pagados.
# Aquí, por simplicidad, imputaremos con 0 (asumiendo que si no hay registro, no se considera mora activa para este promedio)
df_clientes_agg_final['Dias_Pago_Vs_Oportuno_Promedio'].fillna(0, inplace=True)

# Imputar NaNs en Antiguedad_Cliente_Dias y Tiempo_Desde_Ult_Revision_Dias (si las fechas originales eran NaT)
df_clientes_agg_final['Antiguedad_Cliente_Dias'].fillna(df_clientes_agg_final['Antiguedad_Cliente_Dias'].median(), inplace=True)
df_clientes_agg_final['Tiempo_Desde_Ult_Revision_Dias'].fillna(df_clientes_agg_final['Tiempo_Desde_Ult_Revision_Dias'].median(), inplace=True)

print("--- df_clientes_agg_final (primeras filas) ---")
print(df_clientes_agg_final.head())
print("\n--- Información de df_clientes_agg_final ---")
df_clientes_agg_final.info()
print("\n--- Valores faltantes en df_clientes_agg_final ---")
print(df_clientes_agg_final.isnull().sum())

## 7. Guardado del Dataset de Cliente Agregado

Es una buena práctica guardar este dataset procesado para uso futuro.

In [ ]:
df_clientes_agg_final.to_parquet('df_clientes_ingenieria.parquet', index=False)
print("\nDataset df_clientes_ingenieria.parquet guardado.")

## 8. Conclusiones y Próximos Pasos

En este notebook de ingeniería de datos, hemos:
1.  Agregado datos transaccionales para crear un perfil a nivel de cliente, capturando su comportamiento de consumo y pago.
2.  Enriquecido estos perfiles con información demográfica y contractual estática.
3.  Calculado nuevas características basadas en fechas, como la antigüedad del cliente.
4.  Codificado variables categóricas para su uso en modelos.
5.  Realizado una limpieza final de los datos, manejando valores faltantes.

El DataFrame resultante, `df_clientes_agg_final` (guardado como `df_clientes_ingenieria.parquet`), ahora contiene una vista consolidada y rica de cada cliente, lista para ser utilizada en:
* **Modelos Predictivos:** Como predicción de abandono (churn), predicción del valor de vida del cliente (CLV), propensión a comprar nuevos servicios, etc.
* **Segmentación Avanzada:** Aplicar algoritmos de clustering más sofisticados o usar estas características para un perfilado más profundo de los segmentos identificados previamente.
* **Análisis de Negocio:** Generar insights para la toma de decisiones estratégicas.

La ingeniería de características es un paso crucial y a menudo iterativo en el proceso de ciencia de datos. Las características creadas aquí son un buen punto de partida, y podrían expandirse aún más según las necesidades específicas del problema de negocio.